In [22]:
import geopandas as gpd
import pandas as pd
import momepy
import os
import pickle
from shapely.ops import nearest_points
import dask
from dask.diagnostics import ProgressBar
import momepy

In [23]:
local_crs = 27700
place = "test_OS"
lat = 55.86421405612109
lng = -4.251846930489373
country = "UK"
crs=4326
radius=1
n=0

In [25]:
buildings = gpd.read_parquet(f"../output/{place}/buildings_morphometric_p3.pq")
tessellation = gpd.read_parquet(f"../output/{place}/tessellation_morphometric_p3.pq")
edges = gpd.read_parquet(f"../output/{place}/edges_p4.pq")
nodes = gpd.read_parquet(f"../output/{place}/nodes_p4.pq")
stroke = gpd.read_parquet(f"../output/{place}/stroke_p3.pq")
blocks = gpd.read_parquet(f"../output/{place}/blocks_morphometric_p4.5.pq")

In [27]:
tessellation_no_geo = tessellation.drop(columns="geometry")

In [28]:
joint_df = pd.merge(buildings, tessellation_no_geo, on='uID')

In [29]:
edges["edge_ID"] = edges.index
nodes["node_ID"] = nodes.index

In [30]:
joint_df["centroid"] = joint_df.geometry.centroid

In [31]:
from dask.distributed import Client, LocalCluster

daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

c:\Users\reube\anaconda3\envs\nbprocessor\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63535 instead
  warnings.warn(
2024-01-31 17:43:24,279 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 63.92 GiB
2024-01-31 17:43:24,302 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 63.92 GiB
2024-01-31 17:43:24,320 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 63.92 GiB
2024-01-31 17:43:24,343 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 63.92 GiB
2024-01-31 17:43:24,364 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 63.92 GiB
2024-01-31 17:43:24,395 - distributed.nanny.memory - WARNING - Ign

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:63535/status,
Dashboard: http://127.0.0.1:63535/status,Workers: 8
Total threads: 16,Total memory: 511.38 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63539,Workers: 8
Dashboard: http://127.0.0.1:63535/status,Total threads: 16
Started: Just now,Total memory: 511.38 GiB
Comm: tcp://127.0.0.1:63575,Total threads: 2
Dashboard: http://127.0.0.1:63578/status,Memory: 63.92 GiB
Nanny: tcp://127.0.0.1:63542,


In [32]:
import geopandas as gpd
from shapely.ops import nearest_points
from tqdm import tqdm

# Assume buildings and nodes are already loaded GeoDataFrames
# where 'buildings' has polygon geometries and 'nodes' has point geometries.

# Step 2: Compute centroids of buildings
joint_df["centroid"] = joint_df.geometry.centroid

# Step 3: Create spatial index for nodes
nodes_sindex = nodes.sindex

# Step 4: Find closest node for each building
def find_closest_node(building, nodes_df, nodes_sindex):
    # Find nearest point
    nearest_idx = list(nodes_sindex.nearest(building['centroid']))[1][0]
    return nearest_idx

# Apply the function to each building with progress tracking
tqdm.pandas(desc="Finding closest nodes")
joint_df['closest_node'] = joint_df.progress_apply(
    find_closest_node, nodes_df=nodes, nodes_sindex=nodes_sindex, axis=1
)

Finding closest nodes: 100%|██████████| 828537/828537 [00:46<00:00, 17873.23it/s]


In [33]:
# Assuming that the index of the nodes GeoDataFrame is meaningful and unique
joint_df = joint_df.merge(nodes, left_on='closest_node', right_index=True, how='left', suffixes=('', '_node'))

In [34]:

# Step 3: Create spatial index for nodes
edges_sindex = edges.sindex

# Apply the function to each building with progress tracking
tqdm.pandas(desc="Finding closest nodes")
joint_df['closest_edge'] = joint_df.progress_apply(
    find_closest_node, nodes_df=edges, nodes_sindex=edges_sindex, axis=1
)

Finding closest nodes: 100%|██████████| 828537/828537 [01:05<00:00, 12630.83it/s]


In [35]:
# Assuming that the index of the nodes GeoDataFrame is meaningful and unique
joint_df = joint_df.merge(edges, left_on='closest_edge', right_index=True, how='left', suffixes=('', '_edge'))

In [36]:

# Step 3: Create spatial index for nodes
stroke_sindex = stroke.sindex

# Apply the function to each building with progress tracking
tqdm.pandas(desc="Finding closest stroke")
joint_df['closest_stroke'] = joint_df.progress_apply(
    find_closest_node, nodes_df=stroke, nodes_sindex=stroke_sindex, axis=1
)

Finding closest stroke: 100%|██████████| 828537/828537 [01:09<00:00, 11903.13it/s]


In [38]:
# Assuming that the index of the nodes GeoDataFrame is meaningful and unique
joint_df = joint_df.merge(stroke, left_on='closest_stroke', right_index=True, how='left', suffixes=('', '_stroke'))

In [39]:
file_path = f"../output/{place}/queen_3.pkl"

if not os.path.exists(file_path):
    result = "File not found."
else:
    # Load the pickle file
    with open(file_path, 'rb') as file:
        queen_3 = pickle.load(file)
    result = "File loaded successfully."


In [41]:
# # Drop columns containing '_weight' in their names
# columns_to_drop = joint_df.filter(like='_weight').columns
# joint_df.drop(columns=columns_to_drop, inplace=True)

In [43]:
# Drop columns containing '_weight' in their names
columns_to_drop = joint_df.filter(like='bID').columns
joint_df.drop(columns=columns_to_drop, inplace=True)

In [44]:
joint_df.columns

Index(['uID', 'geometry', 'height', 'buildings_neighbours_200',
       'building_area', 'building_height', 'volume', 'building_formfactor',
       'building_circular_compactness', 'building_elongation',
       'building_squareness', 'building_eri', 'building_orientation',
       'building_neighbour_dist',
       'building_neighbourhood_interbuilding_distance', 'building_corners',
       'buildings_shared_walls_ratio', 'buildings_centroidcornermean',
       'buildings_centroidcornerstd', 'building_cell_alignment',
       'building_alignment', 'building_perimeter_wall', 'building_adjacency',
       'tess_compactnessweightedaxis', 'tess_area', 'tessellation_neighbours',
       'tess_convexity', 'tess_covered_area', 'tess_orientation',
       'tess_rea_theil', 'tess_area_ratio', 'tess_circularCompactness',
       'tess_block_count', 'centroid', 'closest_node', 'nodeID',
       'geometry_node', 'node_degree', 'meshedness_eucl_500', 'clustering',
       'meshedness_topo_5', 'closeness_cluste

In [45]:
contextual_df = pd.DataFrame()

In [46]:
adjlist = queen_3.to_adjlist()

In [47]:
joint_df_values = joint_df.drop(columns = ['uID', 'geometry', 'centroid', 'closest_node', 'nodeID', 'geometry_node', 'node_ID', 'closest_edge', 'closest_stroke', 'n_segments', 'geometry_stroke', 'geometry_edge', ])

In [48]:
import pandas as pd
import dask
from dask.diagnostics import ProgressBar
import numpy as np

# Assuming client, joint_df_values, and adjlist are already defined

# Scatter joint_df_values and adjlist to all workers
joint_df_values_future, adjlist_future = client.scatter([joint_df_values, adjlist], broadcast=True)

# Function to process each row, using the scattered dataframes
# Function to process each row, modified to use the scattered dataframes
@dask.delayed
def process_row(index, row, adjlist_scattered, joint_df_values_scattered):
    # this is a function to compute a measure of diversity (Theil index)
    def theil(y):
        y = np.array(y)
        n = len(y)
        plus = y + np.finfo('float').tiny * (y == 0)  # can't have 0 values
        yt = plus.sum(axis=0)
        s = plus / (yt * 1.0)
        lns = np.log(n * s)
        slns = s * lns
        t = sum(slns)
        return t

    neigh_uID = adjlist_scattered[adjlist_scattered['focal'] == index].neighbor.tolist()
    neigh_uID.append(index)
    
    neigh_df = joint_df_values_scattered.loc[neigh_uID]
    
    statistics = {}
    for column in neigh_df.columns:
        # Median
        median_value = neigh_df[column].median()
        statistics[f'{column}_median'] = median_value
        
        # Interquartile Range
        Q1 = neigh_df[column].quantile(0.25)
        Q3 = neigh_df[column].quantile(0.75)
        IQR = Q3 - Q1
        statistics[f'{column}_IQR'] = IQR
        statistics[f'{column}_25_75_mean'] =  np.mean(neigh_df[column][(neigh_df[column] >= Q1) & (neigh_df[column] <= Q3)])
        
        P10 = np.percentile(neigh_df[column], 10)
        P90 = np.percentile(neigh_df[column], 90)
        data_10_90 = neigh_df[column][(neigh_df[column] >= P10) & (neigh_df[column] <= P90)]
        
        
        statistics[f'{column}_10_90_Theil'] = theil(data_10_90)
        
    return statistics

# Function to process a chunk of rows
def process_chunk(chunk, adjlist_scattered, joint_df_values_scattered):
    tasks = [process_row(index, row, adjlist_scattered, joint_df_values_scattered) for index, row in chunk.iterrows()]
    return dask.compute(*tasks)

# Main processing loop
chunk_size = 8000
chunks = [joint_df_values.iloc[i:i+chunk_size] for i in range(0, joint_df_values.shape[0], chunk_size)]

import glob

# Directory where the files are stored
output_directory = f"../output/{place}"
os.makedirs(output_directory, exist_ok=True)

# Pattern to match the files to be deleted
file_pattern = os.path.join(output_directory, "percentile_chunk_*.pq")

# # Find and delete files matching the pattern
# for file in glob.glob(file_pattern):
#     os.remove(file)
#     print(f"Deleted file: {file}")
    
for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    
    if i < n:
        continue
    else:
        with ProgressBar():
            result = process_chunk(chunk, adjlist_future, joint_df_values_future)
            df = pd.DataFrame(result)

            # Save each chunk to a Parquet file
            output_directory = f"../output/{place}"
            os.makedirs(output_directory, exist_ok=True)
            file_path = os.path.join(output_directory, f"percentile_chunk_{i}.pq")
            df.to_parquet(file_path)

            print(f"Chunk {i} saved to {file_path}")

  1%|          | 1/104 [06:13<10:40:28, 373.09s/it]

Chunk 0 saved to ../output/Glasgow_OS\percentile_chunk_0.pq


  2%|▏         | 2/104 [12:07<10:15:38, 362.14s/it]

Chunk 1 saved to ../output/Glasgow_OS\percentile_chunk_1.pq


  3%|▎         | 3/104 [17:56<9:59:22, 356.07s/it] 

...

Chunk 20 saved to ../output/Glasgow_OS\percentile_chunk_20.pq


 21%|██        | 22/104 [2:08:59<7:54:42, 347.35s/it]

Chunk 21 saved to ../output/Glasgow_OS\percentile_chunk_21.pq


 22%|██▏       | 23/104 [2:14:37<7:45:22, 344.73s/it]

Chunk 22 saved to ../output/Glasgow_OS\percentile_chunk_22.pq
